In [ ]:
# Take the hiking weather data and analyze it

In [ ]:
# Import relevant libaries
import json, requests, regex, csv, pandas as pd

In [ ]:
# Allow users to enter their own hikes and coordinates.
coord_dct={}
while True:
    print("Please enter the name of the hike.")
    hike_name=input()
    while True:
        print("Please enter the latitude and longitude coordinates of the hike.\nYou can use https://www.google.com/maps to find the coordinates.")
        hike_coord=input()
        if regex.search(r'^\d{1,2}.\d{4,},\s*-{,1}\d{1,3}.\d{4,}$',hike_coord):
            break
    coord_dct[hike_name]=hike_coord.split(",")[0]+","+hike_coord.split(",")[1].replace(" ","")
    while True:
        print("Do you want to enter another hike? Please enter \"yes\" or \"no\".")
        answer=input()
        if answer in ["yes","no"]:
            break
    if answer=="no":
        break

In [ ]:
# Download the forecast links from the National Weather Service's API using the coord_lst
fc_links={}
for h in coord_dct:
    try:
        response=requests.get('https://api.weather.gov/points/'+coord_dct[h])
        response.raise_for_status()
        data=json.loads(response.text)
        fc_links[h]=data['properties']['forecast']
    except:
        print(f"There was an error with {h}.")

In [ ]:
header=['hikeName','time','temperature','probabilityOfPrecipitation','windSpeed','detailedForecast']

In [ ]:
# Create new csv
with open('weatherforHikes.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    f.close()

In [ ]:
# Write each day of forecast for each hike to csv
for l in fc_links:
    try: 
        r2=requests.get(fc_links[l])
        r2.raise_for_status()
        d2=json.loads(r2.text)
        i=0
        while i<9:
            with open('weatherforHikes.csv', 'a', newline='\n') as f:
                writer = csv.writer(f)
                writer.writerow([l,d2['properties']['periods'][i]['name'],d2['properties']['periods'][i]['temperature'],d2['properties']['periods'][i]['probabilityOfPrecipitation']['value'],d2['properties']['periods'][i]['windSpeed']+" "+d2['properties']['periods'][i]['windDirection'],d2['properties']['periods'][i]['detailedForecast']])
                f.close()
            i+=1
    except:
        print(f"There was an error with {l}.")
    print("\n")

In [ ]:
# Import csv into dataframe
df=pd.read_csv('weatherforHikes.csv')

In [ ]:
df